In [121]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier



In [7]:
df = pd.read_csv('X_TRAIN.csv') # need to remove uid and poverty_score_y

In [8]:
test_df = pd.read_csv('X_TEST.csv')

In [9]:
# poverty = poverty['poverty_score_y'].to_numpy()

In [10]:
# forgot about this. for ocd reasons let
df['attended_school'] = df['attended_school'].eq(1)
df = df.drop(columns=['finish_school_age'])

test_df['attended_school'] = test_df['attended_school'].eq(1)
test_df = test_df.drop(columns=['finish_school_age'])


In [11]:
df.head()

,uid,literate,attended_school,highest_school_lvl,preschool,school_transportation_time,school_transportation_cost,school_expenses,is_student,not_attend_reason1_2,...,marital_status_4,marital_status_5,poverty_score_y,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
0,441_2_3,True,True,2.0,0.0,40.0,0.0,140000.0,True,False,...,False,False,4.0,True,19,0,False,True,2.19,0
1,647_7_1,True,True,2.0,0.0,15.0,0.0,75000.0,True,False,...,False,False,4.0,True,19,0,False,True,2.19,0
2,756_4_1,True,True,2.0,0.0,6.0,0.0,100000.0,True,False,...,False,False,6.0,False,19,0,False,False,3.00,2
3,25_4_3,True,True,5.0,2.0,20.0,0.0,730000.0,True,False,...,False,False,4.0,True,20,0,False,True,2.19,0
4,132_6_3,True,True,2.0,0.0,30.0,0.0,170000.0,True,False,...,True,False,3.0,True,20,0,False,True,2.19,0


In [12]:
test_df.head()

,uid,literate,attended_school,highest_school_lvl,preschool,school_transportation_time,school_transportation_cost,school_expenses,is_student,not_attend_reason1_2,...,marital_status_3,marital_status_4,marital_status_5,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
0,648_6_4,True,True,5.0,3.0,15.0,0.0,130000.0,True,False,...,False,False,False,True,19,0,False,True,2.2,0
1,756_3_3,True,True,5.0,0.0,20.0,20000.0,230000.0,True,False,...,False,False,False,False,19,0,False,True,2.2,0
2,164_8_3,True,True,5.0,3.0,15.0,0.0,280000.0,True,False,...,False,False,False,False,21,0,False,True,2.2,0
3,375_4_4,True,True,5.0,0.0,15.0,0.0,113000.0,True,False,...,False,False,False,False,23,0,True,False,3.0,2
4,403_9_1,True,True,5.0,0.0,120.0,100000.0,100000.0,True,False,...,False,False,False,True,23,0,True,False,3.0,1


# Optional. Split into groups.

In [14]:
group1 = df['attended_school'].eq(False)
g1 = df[group1]
school_variates = [
    'highest_school_lvl', 'preschool', 'school_transportation_time', 'school_transportation_cost',
    'school_expenses', 'is_student', 'not_attend_reason1_2', 'not_attend_reason1_3', 'not_attend_reason1_4', 
    'not_attend_reason1_5', 'not_attend_reason1_6', 'not_attend_reason1_7', 'not_attend_reason1_8', 'not_attend_reason1_9', 
    'not_attend_reason1_10', 'not_attend_reason1_11', 'not_attend_reason1_12', 'not_attend_reason1_13', 'not_attend_reason1_14',
    'not_attend_reason2_2', 'not_attend_reason2_3', 'not_attend_reason2_4', 
    'not_attend_reason2_5', 'not_attend_reason2_6', 'not_attend_reason2_7', 'not_attend_reason2_8', 'not_attend_reason2_9', 
    'not_attend_reason2_10', 'not_attend_reason2_11', 'not_attend_reason2_12', 'not_attend_reason2_13', 'not_attend_reason2_14',
    'school_type_2', 'school_type_3', 
    'transport_2', 'transport_3', 'transport_4',
    'attended_school'
]
g1 = g1.drop(columns=school_variates)

In [15]:
group1_test = test_df['attended_school'].eq(False)
g1_test = test_df[group1_test]
school_variates = [
    'highest_school_lvl', 'preschool', 'school_transportation_time', 'school_transportation_cost',
    'school_expenses', 'is_student', 'not_attend_reason1_2', 'not_attend_reason1_3', 'not_attend_reason1_4', 
    'not_attend_reason1_5', 'not_attend_reason1_6', 'not_attend_reason1_7', 'not_attend_reason1_8', 'not_attend_reason1_9', 
    'not_attend_reason1_10', 'not_attend_reason1_11', 'not_attend_reason1_12', 'not_attend_reason1_13', 'not_attend_reason1_14',
    'not_attend_reason2_2', 'not_attend_reason2_3', 'not_attend_reason2_4', 
    'not_attend_reason2_5', 'not_attend_reason2_6', 'not_attend_reason2_7', 'not_attend_reason2_8', 'not_attend_reason2_9', 
    'not_attend_reason2_10', 'not_attend_reason2_11', 'not_attend_reason2_12', 'not_attend_reason2_13', 'not_attend_reason2_14',
    'school_type_2', 'school_type_3', 
    'transport_2', 'transport_3', 'transport_4',
    'attended_school'
]
g1_test = g1_test.drop(columns=school_variates)

In [16]:
group2 = df['attended_school'].eq(True) & (df['not_attend_reason2_13'].eq(True) | df['not_attend_reason1_13'].eq(True))
g2 = df[group2]

In [17]:
group2_test = test_df['attended_school'].eq(True) & (test_df['not_attend_reason2_13'].eq(True) | test_df['not_attend_reason1_13'].eq(True))
g2_test = test_df[group2_test]

In [18]:
g2 = g2.drop(columns=['attended_school', 'not_attend_reason2_13', 'not_attend_reason1_13'])

In [19]:
g2_test = g2_test.drop(columns=['attended_school', 'not_attend_reason2_13', 'not_attend_reason1_13'])

In [20]:
group3 = df['attended_school'].eq(True) & df['not_attend_reason2_13'].eq(False) & df['not_attend_reason1_13'].eq(False)
g3 = df[group3]

In [21]:
g3 = g3.drop(columns=['attended_school', 'not_attend_reason2_13', 'not_attend_reason1_13'])

In [22]:
group3_test = test_df['attended_school'].eq(True) & test_df['not_attend_reason2_13'].eq(False) & test_df['not_attend_reason1_13'].eq(False)
g3_test = test_df[group3_test]

In [23]:
g3_test = g3_test.drop(columns=['attended_school', 'not_attend_reason2_13', 'not_attend_reason1_13'])

# Models groups

## G1

In [26]:
g1

,uid,literate,fam_role_2,fam_role_3,fam_role_4,fam_role_5,fam_role_6,fam_role_7,fam_role_8,fam_role_9,...,marital_status_4,marital_status_5,poverty_score_y,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
32,773_5_3,False,False,False,False,False,False,False,False,False,...,False,True,1.0,True,24,0,False,False,1.00,0
39,415_7_1,False,False,False,False,False,False,False,False,False,...,False,False,3.0,True,30,0,False,True,2.19,0
44,89_4_1,True,False,False,False,False,False,False,False,False,...,False,False,3.0,True,35,0,True,True,2.19,0
47,794_1_1,False,False,False,False,False,False,False,False,False,...,False,False,5.0,True,36,0,True,True,2.19,0
48,145_7_1,True,True,False,False,False,False,False,False,False,...,False,False,7.0,False,37,12,False,False,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,76_2_1,False,False,False,False,False,False,False,False,False,...,False,False,3.0,False,89,0,False,False,1.50,0
185,272_4_1,False,False,False,False,False,False,False,False,False,...,False,False,6.0,True,90,0,True,False,1.00,0
186,505_3_1,False,False,False,False,False,False,False,False,False,...,False,False,3.0,True,90,0,False,False,1.00,0
187,30_7_1,False,False,False,False,False,False,False,False,False,...,False,False,4.0,True,91,0,True,False,1.00,0


In [27]:
X = g1.drop(columns=['poverty_score_y', 'uid'])
y = g1['poverty_score_y'].to_numpy()


### Random Forest

In [29]:
# param_grid = {
#     'n_estimators': [10, 50, 100],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10]
# }
# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# grid_search.fit(X, y)

# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

### kNN***

In [31]:
knn = KNeighborsClassifier(n_neighbors=25)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# Print the accuracy for each fold
print("Cross-validation scores:", cv_scores)
print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



Cross-validation scores: [0.21428571 0.28571429 0.21428571 0.32142857 0.22222222]
Average cross-validation accuracy: 0.25


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


### Logistic

In [33]:
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# # Perform cross-validation with 5 folds
# cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# # Print cross-validation scores
# print("Cross-validation scores:", cv_scores)
# print("Mean accuracy:", cv_scores.mean())

### G1 real predictions

In [35]:
g1_test.head(3)

,uid,literate,fam_role_2,fam_role_3,fam_role_4,fam_role_5,fam_role_6,fam_role_7,fam_role_8,fam_role_9,...,marital_status_3,marital_status_4,marital_status_5,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
9,93_6_1,True,False,False,False,False,False,False,False,False,...,False,True,False,True,39,0,False,False,2.0,0
10,15_11_2,False,True,False,False,False,False,False,False,False,...,False,False,False,True,40,0,True,True,2.2,0
11,448_3_1,False,False,False,False,False,False,False,False,False,...,False,True,False,True,41,0,True,False,2.0,2


In [36]:
# get X_test
X_TEST = g1_test.drop(columns=['uid'])

In [37]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X, y)
y_pred_g1 = knn.predict_proba(X_TEST)

In [125]:
y_pred_g1 = np.append(y_pred_g1, np.zeros((y_pred_g1.shape[0], 1)), axis=1)
y_pred_g1

array([[0.08, 0.08, 0.36, 0.16, 0.2 , 0.08, 0.04, 0.  , 0.  , 0.  ],
       [0.12, 0.08, 0.36, 0.16, 0.2 , 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.12, 0.12, 0.32, 0.16, 0.2 , 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.12, 0.12, 0.32, 0.16, 0.2 , 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.12, 0.12, 0.28, 0.2 , 0.2 , 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.16, 0.12, 0.28, 0.2 , 0.16, 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.28, 0.12, 0.16, 0.2 , 0.16, 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.24, 0.12, 0.12, 0.2 , 0.2 , 0.12, 0.  , 0.  , 0.  , 0.  ],
       [0.24, 0.16, 0.12, 0.2 , 0.16, 0.12, 0.  , 0.  , 0.  , 0.  ],
       [0.24, 0.16, 0.12, 0.16, 0.2 , 0.12, 0.  , 0.  , 0.  , 0.  ],
       [0.12, 0.2 , 0.04, 0.32, 0.2 , 0.08, 0.04, 0.  , 0.  , 0.  ],
       [0.12, 0.2 , 0.12, 0.24, 0.2 , 0.08, 0.04, 0.  , 0.  , 0.  ],
       [0.08, 0.24, 0.16, 0.24, 0.2 , 0.04, 0.04, 0.  , 0.  , 0.  ],
       [0.08, 0.24, 0.16, 0.24, 0.2 , 0.04, 0.04, 0.  , 0.  , 0.  ],
       [0.08, 0.2 , 0.12, 0.2 , 0.

In [127]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y_pred_g1 = pd.DataFrame(y_pred_g1, columns=cols)
y_pred_g1.insert(0, 'psu_hh_idcode', g1_test['uid'])
y_pred_g1

,psu_hh_idcode,subjective_poverty_1,subjective_poverty_2,subjective_poverty_3,subjective_poverty_4,subjective_poverty_5,subjective_poverty_6,subjective_poverty_7,subjective_poverty_8,subjective_poverty_9,subjective_poverty_10
0,NaN,0.08,0.08,0.36,0.16,0.20,0.08,0.04,0.00,0.00,0.0
1,NaN,0.12,0.08,0.36,0.16,0.20,0.08,0.00,0.00,0.00,0.0
2,NaN,0.12,0.12,0.32,0.16,0.20,0.08,0.00,0.00,0.00,0.0
3,NaN,0.12,0.12,0.32,0.16,0.20,0.08,0.00,0.00,0.00,0.0
4,NaN,0.12,0.12,0.28,0.20,0.20,0.08,0.00,0.00,0.00,0.0
5,NaN,0.16,0.12,0.28,0.20,0.16,0.08,0.00,0.00,0.00,0.0
6,NaN,0.28,0.12,0.16,0.20,0.16,0.08,0.00,0.00,0.00,0.0
7,NaN,0.24,0.12,0.12,0.20,0.20,0.12,0.00,0.00,0.00,0.0
8,NaN,0.24,0.16,0.12,0.20,0.16,0.12,0.00,0.00,0.00,0.0
9,93_6_1,0.24,0.16,0.12,0.16,0.20,0.12,0.00,0.00,0.00,0.0


## G3

In [57]:
X = g3.drop(columns=['poverty_score_y', 'uid'])
y = g3['poverty_score_y'].to_numpy()


### Random Forest

In [42]:
# param_grid = {
#     'n_estimators': [10, 50, 100],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10]
# }
# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# grid_search.fit(X, y)

# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

### kNN

In [44]:
# knn = KNeighborsClassifier(n_neighbors=20)

# # Perform 5-fold cross-validation
# cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# # Print the accuracy for each fold
# print("Cross-validation scores:", cv_scores)
# print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



### Logistic

In [46]:
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# # Perform cross-validation with 5 folds
# cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# # Print cross-validation scores
# print("Cross-validation scores:", cv_scores)
# print("Mean accuracy:", cv_scores.mean())

### NN**

In [59]:
nn_model = MLPClassifier(max_iter=1000, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Perform the grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=sgd; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, so

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=adaptive, solver=sgd; total time=   0.1s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.1s
[CV] END activation=relu, 

#### predicting G3 

In [55]:
X_TEST = g3_test.drop(columns=['uid'])


In [79]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), activation='tanh', solver='sgd',  learning_rate= 'constant', random_state=42)

# Train the model
mlp.fit(X, y)

# Predict probabilities for the test set
y_pred_g3 = mlp.predict_proba(X_TEST)


In [81]:
y_pred_g3

array([[0.0568678 , 0.06816417, 0.14677575, ..., 0.02678193, 0.01081904,
        0.00535599],
       [0.08672939, 0.01755272, 0.14300555, ..., 0.05512445, 0.0148165 ,
        0.00359301],
       [0.0568678 , 0.06816417, 0.14677575, ..., 0.02678193, 0.01081904,
        0.00535599],
       ...,
       [0.0568678 , 0.06816417, 0.14677575, ..., 0.02678193, 0.01081904,
        0.00535599],
       [0.0568678 , 0.06816417, 0.14677575, ..., 0.02678193, 0.01081904,
        0.00535599],
       [0.0568678 , 0.06816417, 0.14677575, ..., 0.02678193, 0.01081904,
        0.00535599]])

In [111]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y_pred_g3 = pd.DataFrame(y_pred_g3, columns=cols)
y_pred_g3.insert(0, 'psu_hh_idcode', g3_test['uid'])
y_pred_g3

,psu_hh_idcode,subjective_poverty_1,subjective_poverty_2,subjective_poverty_3,subjective_poverty_4,subjective_poverty_5,subjective_poverty_6,subjective_poverty_7,subjective_poverty_8,subjective_poverty_9,subjective_poverty_10
0,648_6_4,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356
1,756_3_3,0.086729,0.017553,0.143006,0.171392,0.110393,0.303323,0.094071,0.055124,0.014817,0.003593
2,164_8_3,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356
3,375_4_4,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356
4,403_9_1,0.318490,0.024044,0.059142,0.043056,0.033277,0.051768,0.054729,0.029709,0.363870,0.021914
...,...,...,...,...,...,...,...,...,...,...,...
436,784_4_1,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356
437,831_6_2,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356
438,665_5_1,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356
439,12_7_1,0.056868,0.068164,0.146776,0.226106,0.191317,0.167455,0.100357,0.026782,0.010819,0.005356


## G2

In [133]:
X = g2.drop(columns=['poverty_score_y', 'uid'])
y = g2['poverty_score_y'].to_numpy()


### Random Forest**

In [135]:
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

Best parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 50}
Best cross-validation accuracy: 0.21


### kNN

In [ ]:
# knn = KNeighborsClassifier(n_neighbors=50)

# # Perform 5-fold cross-validation
# cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# # Print the accuracy for each fold
# print("Cross-validation scores:", cv_scores)
# print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



### Logistic

In [ ]:
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# # Perform cross-validation with 5 folds
# cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# # Print cross-validation scores
# print("Cross-validation scores:", cv_scores)
# print("Mean accuracy:", cv_scores.mean())

### NN

In [ ]:
# nn_model = MLPClassifier(max_iter=1000, random_state=42)

# # Define the hyperparameter grid
# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
#     'activation': ['relu', 'tanh'],
#     'solver': ['adam', 'sgd'],
#     'learning_rate': ['constant', 'adaptive'],
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# # Perform the grid search
# grid_search.fit(X, y)

# # Print the best parameters and best score
# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation accuracy:", grid_search.best_score_)


# fitting to G2

In [137]:
X_TEST = g2_test.drop(columns=['uid'])


In [150]:
# rf parameters Best parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 50}
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=10, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X, y)

# Make predictions on the test data
y_pred_g2 = rf.predict_proba(X_TEST)


In [152]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y_pred_g2 = pd.DataFrame(y_pred_g2, columns=cols)


In [156]:
g2_test

,uid,literate,highest_school_lvl,preschool,school_transportation_time,school_transportation_cost,school_expenses,is_student,not_attend_reason1_2,not_attend_reason1_3,...,marital_status_3,marital_status_4,marital_status_5,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
6,684_6_2,True,5.0,0.0,56.0,30000.0,320000.0,True,False,False,...,False,False,False,False,24,0,False,True,2.2,0
7,524_3_2,True,5.0,2.0,180.0,16000.0,1600000.0,True,False,False,...,False,False,False,False,25,0,False,False,5.0,2
8,755_8_1,True,6.0,0.0,75.0,0.0,3000000.0,True,False,False,...,False,False,False,False,25,0,False,False,3.5,1
53,152_9_1,True,1.0,0.0,56.0,0.0,531500.0,False,False,False,...,False,False,False,False,74,0,False,False,2.0,0
55,680_1_3,True,5.0,1.0,56.0,0.0,531500.0,False,False,False,...,False,False,False,False,19,0,False,True,2.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,612_1_1,True,2.0,1.0,56.0,0.0,531500.0,False,False,False,...,False,False,False,False,22,0,False,True,2.2,0
1330,738_1_3,True,5.0,0.0,56.0,0.0,531500.0,False,False,False,...,False,False,False,False,22,0,False,True,2.2,0
1331,191_10_2,True,1.0,1.0,56.0,0.0,531500.0,False,False,False,...,False,False,False,False,23,0,True,False,2.5,2
1332,475_9_5,True,2.0,0.0,56.0,0.0,531500.0,False,False,False,...,False,False,False,True,23,0,False,True,2.2,0


In [154]:
y_pred_g2

,subjective_poverty_1,subjective_poverty_2,subjective_poverty_3,subjective_poverty_4,subjective_poverty_5,subjective_poverty_6,subjective_poverty_7,subjective_poverty_8,subjective_poverty_9,subjective_poverty_10
0,0.005763,0.050791,0.198173,0.318574,0.164609,0.123947,0.073916,0.055359,0.006130,0.002737
1,0.016182,0.111103,0.197476,0.175422,0.096142,0.208215,0.095237,0.089081,0.010843,0.000299
2,0.012609,0.095840,0.272877,0.221715,0.161193,0.099328,0.068484,0.058440,0.008717,0.000797
3,0.026605,0.120945,0.098642,0.277773,0.228238,0.142204,0.074642,0.026069,0.003933,0.000949
4,0.042998,0.021495,0.084442,0.177574,0.135546,0.190347,0.196034,0.112697,0.036990,0.001878
...,...,...,...,...,...,...,...,...,...,...
849,0.049898,0.061716,0.130824,0.241135,0.200927,0.163244,0.077325,0.062537,0.011820,0.000574
850,0.014680,0.027415,0.085863,0.222554,0.212306,0.216234,0.106708,0.102438,0.007124,0.004678
851,0.037082,0.081012,0.208717,0.209317,0.206645,0.115370,0.093908,0.039798,0.007888,0.000263
852,0.026280,0.068692,0.139432,0.288287,0.170920,0.175416,0.075589,0.044322,0.007907,0.003154


In [158]:
y_pred_g2.insert(0, 'psu_hh_idcode', g2_test['uid'])
y_pred_g2

,psu_hh_idcode,subjective_poverty_1,subjective_poverty_2,subjective_poverty_3,subjective_poverty_4,subjective_poverty_5,subjective_poverty_6,subjective_poverty_7,subjective_poverty_8,subjective_poverty_9,subjective_poverty_10
0,NaN,0.005763,0.050791,0.198173,0.318574,0.164609,0.123947,0.073916,0.055359,0.006130,0.002737
1,NaN,0.016182,0.111103,0.197476,0.175422,0.096142,0.208215,0.095237,0.089081,0.010843,0.000299
2,NaN,0.012609,0.095840,0.272877,0.221715,0.161193,0.099328,0.068484,0.058440,0.008717,0.000797
3,NaN,0.026605,0.120945,0.098642,0.277773,0.228238,0.142204,0.074642,0.026069,0.003933,0.000949
4,NaN,0.042998,0.021495,0.084442,0.177574,0.135546,0.190347,0.196034,0.112697,0.036990,0.001878
...,...,...,...,...,...,...,...,...,...,...,...
849,228_8_1,0.049898,0.061716,0.130824,0.241135,0.200927,0.163244,0.077325,0.062537,0.011820,0.000574
850,336_3_1,0.014680,0.027415,0.085863,0.222554,0.212306,0.216234,0.106708,0.102438,0.007124,0.004678
851,413_8_1,0.037082,0.081012,0.208717,0.209317,0.206645,0.115370,0.093908,0.039798,0.007888,0.000263
852,430_9_1,0.026280,0.068692,0.139432,0.288287,0.170920,0.175416,0.075589,0.044322,0.007907,0.003154


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!$$$$$$
# Models General case

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Initialize Logistic Regression Model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

# kNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=40)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# Print the accuracy for each fold
print("Cross-validation scores:", cv_scores)
print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



In [ ]:
knn.fit(X_train, y_train)
knn.predict_proba(X_test)

# Random Forest

In [ ]:
# Set up a grid of hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV to search over the hyperparameter grid
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV to the training data
grid_search.fit(X, y)

# Get the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")


In [ ]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy:.2f}")


In [ ]:
# X_test

# Neural network

In [ ]:
from sklearn.neural_network import MLPClassifier

# Initialize the MLPClassifier for a neural network
# 'hidden_layer_sizes' defines the number of neurons in the hidden layers (e.g., 100 neurons in one hidden layer)
# 'max_iter' is the maximum number of iterations for training
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, solver='adam')

# Train the model
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy:.2f}")


# Do it for reals!

In [ ]:
X_test = pd.read_csv('X_TEST.csv')


In [ ]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X, y)

# Make predictions on the test data
y_pred = rf.predict_proba(X_test)

# Evaluate the model using accuracy
# accuracy = accuracy_score(y_test, y_pred)

# print(f"Accuracy on the test set: {accuracy:.2f}")


In [ ]:
y_pred

In [ ]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y = pd.DataFrame(y_pred, columns=cols)

In [ ]:
y

In [ ]:
uids = pd.read_csv('uids.csv')

In [ ]:
ans = pd.concat([uids, y], axis=1)
ans.rename(columns={'uid': 'psu_hh_idcode'}, inplace=True)


In [ ]:
ans

In [ ]:
ans.to_csv('solution.csv', index=False)
